In [7]:

!pip install pandas seaborn matplotlib scipy 

   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/43.6 MB ? eta -:--:--
    --------------------------------------- 0.8/43.6 MB 1.2 MB/s eta 0:00:38
    --------------------------------------- 1.0/43.6 MB 1.1 MB/s eta 0:00:39
   - -------------------------------------- 1.8/43.6 MB 1.8 MB/s eta 0:00:24
   -- ------------------------------------- 2.6/43.6 MB 2.1 MB/s eta 0:00:20
   --- ------------------------------------ 3.9/43.6 MB 2.9 MB/s eta 0:00:14
   ------ --------------------------------- 6.6/43.6 MB 4.0 MB/s eta 0:00:10
   ------- -------------------------------- 8.7/43.6 MB 4.7 MB/s eta 0:00:08
   ------------- ------------------------

In [13]:

import pandas as pd

# Read the CASTransactionSummary CSV file
df = pd.read_csv('CASTransactionSummary.csv')

# Display the first few rows of the dataframe
print(df.head())
df.columns


   Session             Start time         Receipt Time   \
0  4067038  08/01/2023 01:01:13 AM  2023-08-01 01:01:13   
1  4067039  08/01/2023 01:03:42 AM  2023-08-01 01:03:43   
2  4067040  08/01/2023 01:04:16 AM  2023-08-01 01:04:16   
3  4067041  08/01/2023 01:05:43 AM  2023-08-01 01:05:44   
4  4067046  08/01/2023 01:12:30 AM  2023-08-01 01:12:30   

                 End Time       Location    Card/Check No.    First Name   \
0  08/01/2023 01:01:13 AM  Morongo Casino  434256******8498             -   
1  08/01/2023 01:03:43 AM  Morongo Casino  443042******7551  KHALIL ABDEL   
2  08/01/2023 01:04:16 AM  Morongo Casino  551915******8420             -   
3  08/01/2023 01:05:44 AM  Morongo Casino  515676******1782    JOSE JULIO   
4  08/01/2023 01:12:30 AM  Morongo Casino  440066******8319        MI JIN   

   Last Name   VIP Player Name          Status   Processor   Trans Type   \
0           -                 -  Debit Dispense          DS        Debit   
1      RAHMAN                 

Index(['Session', 'Start time ', ' Receipt Time ', ' End Time', ' Location ',
       ' Card/Check No. ', ' First Name ', ' Last Name ', ' VIP Player Name ',
       ' Status ', ' Processor ', ' Trans Type ', ' Card/Check Type ',
       ' Receipt No. ', ' Amount ', ' Fee ', ' Total ', ' Cashier ',
       ' Err/Dec ', ' Auth '],
      dtype='object')

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from scipy import stats

def analyze_transaction_patterns(df):
    """
    Create baseline statistics for normal transaction patterns.
    
    Parameters:
    df: DataFrame with columns [TransactionDate, TransactionType, AccountNumber, Amount]
    
    Returns:
    dict with baseline statistics
    """
   
    df['TransactionDate'] = pd.to_datetime(df[' End Time'])
    df['Hour'] = df['TransactionDate'].dt.hour
    df['DayOfWeek'] = df['TransactionDate'].dt.dayofweek
  
    df.rename(columns={' Card/Check No. ': 'account_number', 
                       ' Amount ': 'amount', 
                       ' Trans Type ': 'trans_type'}, 
              inplace=True)
 
    
    # Enhanced account-level statistics
    account_stats = {
        # Basic transaction patterns
        'tx_count': df.groupby('account_number').size().describe(),
        'max_amount': df.groupby('account_number')['amount'].max().describe(),
        
        # New: Additional statistical measures
        'amount_skewness': df.groupby('account_number')['amount'].agg(stats.skew).describe(),
        'amount_kurtosis': df.groupby('account_number')['amount'].agg(stats.kurtosis).describe(),
        'tx_entropy': df.groupby('account_number')['trans_type'].agg(lambda x: stats.entropy(x.value_counts())).describe(),
     
        
        # Time patterns
        'common_hours': df.groupby('Hour').size().sort_values(ascending=False),
        'common_days': df.groupby('DayOfWeek').size().sort_values(ascending=False),
        
        # Transaction type distribution
        'tx_type_dist': df['trans_type'].value_counts(normalize=True),
        
   }
    
    # Amount patterns
    avg_amount = df.groupby('account_number')['amount'].mean()
    
    account_stats['avg_amount'] = {
        'mean': avg_amount.mean(),
        'std': avg_amount.std(),
        'percentiles': avg_amount.quantile([0.25, 0.5, 0.75, 0.95, 0.99])
    }
    # Enhanced velocity metrics
    velocities = {
        '1H': df.groupby(['account_number', pd.Grouper(key='TransactionDate', freq='1h')]).size(),
        '24H': df.groupby(['account_number', pd.Grouper(key='TransactionDate', freq='24h')]).size(),
        '7D': df.groupby(['account_number', pd.Grouper(key='TransactionDate', freq='7D')]).size()
    }
    
    account_stats['velocity_metrics'] = {
        period: {
            'mean': vel.mean(),
            'std': vel.std(),
            'percentiles': vel.quantile([0.25, 0.5, 0.75, 0.95, 0.99])
        }
        for period, vel in velocities.items()
    }
    
    return account_stats



In [24]:
baseline_stats=analyze_transaction_patterns(df)
baseline_stats

c:\Users\HP\OneDrive\Documents\Eduardo Toledo\ASAI\antimoneylaundering\.venv\Lib\site-packages\pandas\core\groupby\generic.py:324: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  f = lambda x: func(x, *args, **kwargs)
c:\Users\HP\OneDrive\Documents\Eduardo Toledo\ASAI\antimoneylaundering\.venv\Lib\site-packages\pandas\core\groupby\generic.py:324: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  f = lambda x: func(x, *args, **kwargs)


{'tx_count': count    51366.000000
 mean         2.841685
 std          4.793049
 min          1.000000
 25%          1.000000
 50%          1.000000
 75%          3.000000
 max        192.000000
 dtype: float64,
 'max_amount': count    51366.000000
 mean       554.989662
 std        844.900156
 min          7.000000
 25%        120.000000
 50%        300.000000
 75%        500.000000
 max      20000.000000
 Name: amount, dtype: float64,
 'amount_skewness': count    20141.000000
 mean         0.131779
 std          0.689780
 min         -4.694855
 25%          0.000000
 50%          0.000000
 75%          0.506408
 max          8.553677
 Name: amount, dtype: float64,
 'amount_kurtosis': count    20141.000000
 mean        -1.194952
 std          1.506675
 min         -2.000000
 25%         -2.000000
 50%         -1.500000
 75%         -0.960000
 max         75.135978
 Name: amount, dtype: float64,
 'tx_entropy': count    51366.000000
 mean         0.031858
 std          0.134265
 min   

In [41]:
def is_amount_suspicious(transaction, df, baseline_stats):
    # Get account's typical behavior
    account_history = df[df['account_number'] == transaction['account_number']].copy()
    
    # Check if we have enough transactions for meaningful statistics
    if len(account_history) >= 2:
        # Use robust statistics less sensitive to non-normal distributions
        median_amount = account_history['amount'].median()
        q1 = account_history['amount'].quantile(0.25)
        q3 = account_history['amount'].quantile(0.75)
        iqr = q3 - q1
    else:
        # Use baseline statistics if not enough account history
        median_amount = baseline_stats['avg_amount']['percentiles'][0.5]  # median
        q1 = baseline_stats['avg_amount']['percentiles'][0.25]
        q3 = baseline_stats['avg_amount']['percentiles'][0.75]
        iqr = q3 - q1
    
    # Use Modified Z-score with median and IQR instead of mean and std
    # This is more robust for non-normal distributions
    if iqr == 0:
        # If IQR is 0, fall back to percent difference from median
        percent_difference = abs((transaction['amount'] - median_amount) / median_amount) if median_amount != 0 else float('inf')
        is_suspicious = percent_difference > 0.5  # Flag if difference is more than 50%
        score = percent_difference
        method = 'percent_difference'
    else:
        # Calculate modified z-score using median and IQR
        # 0.6745 is used because for normal distributions, IQR/0.6745 ≈ std
        modified_score = 0.6745 * (transaction['amount'] - median_amount) / iqr
        is_suspicious = abs(modified_score) > 3.5  # Slightly higher threshold than regular z-score
        score = modified_score
        method = 'modified_z_score'
    
    if is_suspicious:
        alert = {
            'account': transaction['account_number'],
            'alert_type': 'suspicious_amount',
            'subclass': f'{method}: {score:.2f}',
            'transaction': transaction,
            'reference_stats': {
                'median': median_amount,
                'q1': q1,
                'q3': q3,
                'iqr': iqr
            }
        }
        return (True, alert)
    else:
        return (False, None)       
def is_velocity_suspicious(transaction, account_history):
    """
    Calculate z-scores for transaction velocities in different time windows,
    handling cases where there's no historical data for the transaction period
    """
    # Ensure TransactionDate is datetime
    account_history['TransactionDate'] = pd.to_datetime(account_history['TransactionDate'])
    tx_date = pd.Timestamp(transaction['TransactionDate'])
    
    # Create a temporary DataFrame including the new transaction
    new_tx_df = pd.DataFrame([{
        'TransactionDate': tx_date,
        'Amount': transaction['amount']
    }])
    
    # Combine with historical data
    combined_history = pd.concat([account_history, new_tx_df])
    combined_history = combined_history.sort_values('TransactionDate')
    
    velocities = {
        '1h': combined_history.groupby(pd.Grouper(key='TransactionDate', freq='1h')).size(),
        '24h': combined_history.groupby(pd.Grouper(key='TransactionDate', freq='24h')).size(),
        '7D': combined_history.groupby(pd.Grouper(key='TransactionDate', freq='7D')).size()
    }
    
    results = {}
    
    for window, velocity_series in velocities.items():
        # Get current period
        current_period = tx_date.floor(window)
        
        # Calculate mean and std of historical velocities (excluding current period)
        historical_velocities = velocity_series[velocity_series.index < current_period]
        
        if len(historical_velocities) > 0:
            mean_velocity = historical_velocities.mean()
            std_velocity = historical_velocities.std()
        else:
            # No historical data for this time window
            mean_velocity = 0
            std_velocity = 0
        
        # Get current period's velocity
        current_velocity = velocity_series.get(current_period, 1)  # At least 1 for the new transaction
        
        # Calculate z-score
        if std_velocity == 0:
            # If no variation in historical data or no historical data
            z_score = float('inf') if current_velocity > mean_velocity else 0
        else:
            z_score = (current_velocity - mean_velocity) / std_velocity
        
        results[window] = {
            'z_score': z_score,
            'is_suspicious': abs(z_score) > 3,
            'current_velocity': current_velocity,
            'mean_velocity': mean_velocity,
            'std_velocity': std_velocity,
            'has_history': len(historical_velocities) > 0
        }
    
    return results

def flag_unusual_transactions(transaction, df, baseline_stats):
    """
    Flag potentially unusual transactions based on simple rules
    """
    alerts = []
    account_number = "Card/Check No."
    # Amount thresholds (using 95th percentile)
    amount_threshold = baseline_stats['avg_amount']['percentiles'][0.95]
    
    # Velocity threshold (using 95th percentile)
    velocity_24h_threshold = baseline_stats['velocity_metrics']['24H']['percentiles'][0.95]
    velocity_1h_threshold = baseline_stats['velocity_metrics']['1H']['percentiles'][0.95]
    velocity_7d_threshold = baseline_stats['velocity_metrics']['7D']['percentiles'][0.95]
    
    
    
    
    # Check for unusual amounts
    high_amount_tx = transaction['Amount'] > amount_threshold
        
    # Check for unusual velocity
    new_tx_df = pd.DataFrame([transaction])
    new_tx_df['End Time'] = pd.to_datetime(new_tx_df['TransactionDate'])
   
    account_df = df[df[account_number] == transaction[account_number]].copy()
    account_df = pd.concat([account_df, new_tx_df], ignore_index=True)
   
    # Sort by TransactionDate to ensure proper time-based grouping
    account_df = account_df.sort_values('End Time')
    
    counts_24h = account_df.groupby(pd.Grouper(key='End Time', freq='24h')).size()
    high_velocity_24h = counts_24h[counts_24h > velocity_24h_threshold]
    
    counts_1h = account_df.groupby(pd.Grouper(key='End Time', freq='1h')).size()
    high_velocity_1h = counts_1h[counts_1h > velocity_1h_threshold]
                    
    counts_7d = account_df.groupby(pd.Grouper(key='End Time', freq='7D')).size()
    high_velocity_7d = counts_7d[counts_7d > velocity_7d_threshold]
        
        
    if high_amount_tx:
            alerts.append({
                'account': transaction[account_number],
                'alert_type': 'high_amount',
                'transactions': transaction
            })
            
    if not high_velocity_24h.empty:
        tx_date = pd.Timestamp(transaction['TransactionDate'])
        tx_period = tx_date.floor('24h')
        
        if tx_period in high_velocity_24h.index:
            alerts.append({
                'account': transaction[account_number],
                'alert_type': 'high_velocity_24h',
                'dates': high_velocity_24h.index
            })  
    
    
    if not high_velocity_1h.empty:
        tx_date = pd.Timestamp(transaction['TransactionDate'])
        tx_period = tx_date.floor('1h')
        
        if tx_period in high_velocity_1h.index:
            alerts.append({
                'account': transaction[account_number],
                'alert_type': 'high_velocity_1h',
                'dates': high_velocity_1h.index
            })
    
    if not high_velocity_7d.empty:
        # Convert transaction date to the same frequency as the grouper (7D)
        tx_date = pd.Timestamp(transaction['TransactionDate'])
        # Find the period containing the transaction date
        tx_period = tx_date.floor('7D')
        
        if tx_period in high_velocity_7d.index:
            alerts.append({
                'account': transaction[account_number],
                'alert_type': 'high_velocity_7d',
                'dates': high_velocity_7d.index
            })
            
    return alerts



In [42]:
transaction = {
    'account_number':'481582******3305',
    'amount':20000,
    'TransactionDate': '2023-07-13 01:01:13'
}

alert = is_amount_suspicious( transaction, df, baseline_stats)
print(alert)
result = is_velocity_suspicious(transaction, df)
print(result)

(True, {'account': '481582******3305', 'alert_type': 'suspicious_amount', 'subclass': 'modified_z_score: 133.55', 'transaction': {'account_number': '481582******3305', 'amount': 20000, 'TransactionDate': '2023-07-13 01:01:13'}, 'reference_stats': {'median': np.float64(200.0), 'q1': np.float64(100.0), 'q3': np.float64(200.0), 'iqr': np.float64(100.0)}})
{'1h': {'z_score': np.float64(-0.43627011331319965), 'is_suspicious': np.False_, 'current_velocity': np.int64(28), 'mean_velocity': np.float64(38.12152777777778), 'std_velocity': np.float64(23.200140162962533), 'has_history': True}, '24h': {'z_score': np.float64(-0.94443653818143), 'is_suspicious': np.False_, 'current_velocity': np.int64(618), 'mean_velocity': np.float64(913.3333333333334), 'std_velocity': np.float64(312.7085001412754), 'has_history': True}, '7D': {'z_score': np.float64(-11.86517492887753), 'is_suspicious': np.True_, 'current_velocity': 1, 'mean_velocity': np.float64(6176.0), 'std_velocity': np.float64(520.430590953299),